# Dependencies

In [ ]:
%%capture
!pip install faiss-gpu

In [ ]:
%%capture
!pip install rouge fuzzywuzzy

In [ ]:
%%capture
!pip install unsloth "xformers==0.0.28.post2"
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
import faiss
import pandas as pd
import torch
import json
import random
from sentence_transformers import SentenceTransformer
from unsloth import FastLanguageModel
from transformers import TextStreamer
from datasets import load_dataset
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge
from sklearn.metrics import f1_score
from fuzzywuzzy import fuzz  # fuzzywuzzy để tính Levenshtein similarity


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Tải model fine-tuned

In [ ]:
def load_fast_llm(model_name, max_seq_length=8192, dtype=None, load_in_4bit=True):
    print("Đang tải mô hình FastLanguageModel...")
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=model_name,
        max_seq_length=max_seq_length,
        dtype=dtype,
        load_in_4bit=load_in_4bit,
    )
    FastLanguageModel.for_inference(model)
    text_streamer = TextStreamer(tokenizer)
    return model, tokenizer, text_streamer

# Load mô hình FastLanguageModel
model_name = "MTruc/llama3.2-1b-instruct-orpo-v2"
model, tokenizer, text_streamer = load_fast_llm(model_name)

Đang tải mô hình FastLanguageModel...
==((====))==  Unsloth 2024.11.10: Fast Llama patching. Transformers:4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/45.1M [00:00<?, ?B/s]

Unsloth 2024.11.10 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


# Evaluate without RAG

In [ ]:
alpaca_prompt = """Instruction: {}

### Context:
{}

### Input:
{}

### Response:
{}"""

In [ ]:
test_data = load_dataset("MTruc/vn_qa_finance_orpo")["test"]
i = random.randrange(1, 100)
test_data[i]

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

data-00000-of-00001.arrow:   0%|          | 0.00/4.66M [00:00<?, ?B/s]

data-00000-of-00001.arrow:   0%|          | 0.00/844k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2922 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/516 [00:00<?, ? examples/s]

{'Instruction': 'Bạn là một chuyên gia tài chính, bạn được cung cấp những chỉ số tài chính của một mô hình dự đoán giá và những tài liệu tài chính kèm theo. Người dùng có thể hỏi về những chỉ số này hoặc những câu hỏi khác, nếu người dùng hỏi về chỉ số hoặc thông tin liên quan đến mô hình, hãy dùng số liệu được cung cấp để trả lời, còn nếu người dùng hỏi những câu hỏi khác không liên quan đến mô hình thì không cần dùng những chỉ số tài chính của mô hình.',
 'Context': 'Chỉ số của mô hình:\n Return (Ann.) [%]: 69.39,\n Volatility (Ann.) [%]: 30.3,\n Max. Drawdown [%]: -11.35,\n Sharpe Ratio: 2.29,\n Sortino Ratio: 2.11,\n Calmar Ratio: 6.11,\n Win Rate [%]: 69.3,\n AvgTrade [%]: 0.31,\n Generalization Score: 0.96,\n Tỉ lệ dự đoán gần đây [%]: 40.',
 'Text': 'Phát biểu nào sai về Hợp đồng tương lai trái phiếu chính phủ',
 'Choose': 'Giúp NĐT đối phó hiệu quả với biến động lãi suất ngắn hạn và dài hạn',
 'Reject': 'Giúp phân bổ tài sản, đối phó biến động lãi suất trên thị trường mà không 

In [ ]:
questions = list(test_data['Text'])
true_answers = list(test_data['Choose'])
model_answer=[]

for i in range(len(questions)): #len(questions)
  query = questions[i]
  inputs = tokenizer(
  [
      alpaca_prompt.format(
          test_data[i]['Instruction'], # instruction
          test_data[i]['Context'], # context,
          test_data[i]['Text'], # input,
          "", # output - leave this blank for generation!
      )
  ], return_tensors = "pt").to("cuda")

  output = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

  # Lấy phần phản hồi
  generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
  response_start = generated_text.find("### Phản hồi:") + len("### Phản hồi:")
  response = generated_text[response_start:].strip()
  try:
    # Find the index of "Response:\n"
    response_index = response.index("Response:\n")
    # Extract the substring after "Response:\n"
    extracted_answer = response[response_index + len("Response:\n"):].strip()
    model_answer.append(extracted_answer)
  except ValueError:
    # Handle cases where "Response:\n" is not found
    model_answer.append(response)

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
 Max. Drawdown [%]: -21.59,
 Sharpe Ratio: 0.31,
 Sortino Ratio: 2.07,
 Calmar Ratio: 0.46,
 Win Rate [%]: 53.16,
 AvgTrade [%]: 0.05,
 Generalization Score: -0.06,
 Tỉ lệ dự đoán gần đây [%]: 60.

### Input:
Giá trị lớn nhất của quyền chọn bán kiểu Châu Âu:

### Response:
What is the maximum profit you can make from a buy option with a strike price of 100 and a price of 80?<|eot_id|>
<|begin_of_text|>Instruction: Bạn là một chuyên gia tài chính, bạn được cung cấp những chỉ số tài chính của một mô hình dự đoán giá và những tài liệu tài chính kèm theo. Người dùng có thể hỏi về những chỉ số này hoặc những câu hỏi khác, nếu người dùng hỏi về chỉ số hoặc thông tin liên quan đến mô hình, hãy dùng số liệu được cung cấp để trả lời, còn nếu người dùng hỏi những câu hỏi khác không liên quan đến mô hình thì không cần dùng những chỉ số tài chính của mô hình.

### Context:
Chỉ số của mô hình:
 Return (Ann.) [%]: 0.2,
 Volatility (Ann.) [%]: 

In [ ]:
model_answer

['8 USD',
 "Chỉ số '41.88' cho thấy mô hình dự đoán giá có độ tin cậy trung bình trên 41%. Điều này có ý nghĩa gì đối với khả năng tin cậy của mô hình trong việc đưa ra dự đoán chính xác.",
 'Phát biểu "Hợp đồng tương lai phải trả 10 năm sau khi phát hành" sai vì Hợp đồng tương lai có thời hạn giao dịch 1 ngày, nhưng trái phiếu chính phủ được phát hành với kỳ hạn 10 năm.\n\n### Response details:\nChỉ số Generalization Score của 0.96 cho thấy mô hình có khả năng dự đoán hợp đồng tương lai chính xác 96% trong các lần mới. Chỉ số này được sử dụng để đánh giá độ chính xác của mô hình dự đoán hợp đồng tương lai. Một giá trị Generalization Score cao cho thấy mô hình có khả năng dự đoán',
 'Chỉ số Sharpe Ratio là tỷ lệ so sánh giữa lợi nhuận thặng dư (Return) với rủi ro (Volatility) của một mô hình dự đoán giá. Nó giúp người dùng hiểu rõ hơn về hiệu quả của mô hình trong việc tạo ra lợi nhuận với mức độ chấp nhận được rủi ro. Chỉ số này quan trọng vì nó cho thấy liệu mô hình có tạo ra lợi nhu

In [ ]:
def calculate_similarity(pred, true):
    return fuzz.ratio(pred, true) / 100  # Tỉ lệ tương đồng (0-1)

# Chuyển ngưỡng tương đồng thành nhãn nhị phân (giống hoặc không giống)
similarity_threshold = 0.5
binary_predictions = [1 if calculate_similarity(p, t) >= similarity_threshold else 0
                      for p, t in zip(model_answer, true_answers)]
binary_true_answers = [1] * len(true_answers)

# Tính F1-score
f1 = f1_score(binary_true_answers, binary_predictions)
print("F1-score:", f1)

F1-score: 0.10642201834862386


In [ ]:
# Hàm tính toán các chỉ số đánh giá
def evaluate_model(predictions, true_answers):
    # Filter out None values from predictions and true_answers
    # Updated filtering to handle None in both lists, and ensure indices match
    valid_indices = [i for i, (pred, true) in enumerate(zip(predictions, true_answers))
                     if pred is not None and true is not None]
    filtered_predictions = [predictions[i] for i in valid_indices]
    filtered_true_answers = [true_answers[i] for i in valid_indices]

    # Proceed with evaluation using filtered data, only if both lists have valid data
    if not filtered_predictions or not filtered_true_answers:
        print("Warning: No valid data for evaluation after filtering.")
        return {} # or some default values

    bleu_scores = [sentence_bleu([true.split()], pred.split()) for true, pred in zip(filtered_true_answers, filtered_predictions)]
    avg_bleu = sum(bleu_scores) / len(bleu_scores) if bleu_scores else 0  # Handle empty list

    rouge = Rouge()
    rouge_scores = rouge.get_scores(filtered_predictions, filtered_true_answers, avg=True)

    return {
        'bleu': avg_bleu,
        'rouge': rouge_scores
    }

# Đánh giá model DPO
results = evaluate_model(model_answer, true_answers)
results['f1']=f1
print("Results:", results)

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Results: {'bleu': 0.04167949707799821, 'rouge': {'rouge-1': {'r': 0.2783074971726522, 'p': 0.22133480865242916, 'f': 0.23154457026745276}, 'rouge-2': {'r': 0.1313639473767645, 'p': 0.0984707761621934, 'f': 0.10437180475224489}, 'rouge-l': {'r': 0.24926000653602706, 'p': 0.19858897911048592, 'f': 0.20719320499085905}}, 'f1': 0.10642201834862386}


In [ ]:
with open('evaluation_results.json', 'w') as file: json.dump(results, file)

# Evaluate with RAG

In [ ]:
# Load FAISS index và metadata
def load_faiss_index(faiss_path, metadata_path):
    print("Đang tải FAISS index và metadata...")
    index = faiss.read_index(faiss_path)
    metadata = pd.read_csv(metadata_path).to_dict(orient='records')
    return index, metadata

# Truy xuất tài liệu từ vector database
def retrieve_context(query, index, metadata, model, k=5):
    query_embedding = model.encode([query], show_progress_bar=False)[0]
    distances, indices = index.search(query_embedding.reshape(1, -1), k)
    results = [metadata[i]['chunk_content'] for i in indices[0]]
    return "\n".join(results)

# Pipeline trả lời câu hỏi
def rag_llm_pipeline(test_data, model, retrieval_model, tokenizer, index, metadata, alpaca_prompt, k=5):
    # Truy xuất context
    context = retrieve_context(question, index, metadata, retrieval_model, k)

    if type(test_data) is str:
      # Tạo input cho mô hình LLM
      inputs = tokenizer(
          [
              alpaca_prompt.format(
                  "Bạn là một chuyên gia tài chính, bạn được cung cấp những chỉ số tài chính của một mô hình dự đoán giá và những tài liệu tài chính kèm theo. Người dùng có thể hỏi về những chỉ số này hoặc những câu hỏi khác, nếu người dùng hỏi về chỉ số hoặc thông tin liên quan đến mô hình, hãy dùng số liệu được cung cấp để trả lời, còn nếu người dùng hỏi những câu hỏi khác không liên quan đến mô hình thì không cần dùng những chỉ số tài chính của mô hình.",
                  context,
                  test_data,
                  ""  # Output để trống
              )
          ], return_tensors="pt").to("cuda")
    else:
      inputs = tokenizer(
        [
            alpaca_prompt.format(
                test_data['Instruction'], # instruction
                test_data['Context']+ ' ' + context, # context,
                test_data['Text'], # question,
                "", # output - leave this blank for generation
            )
        ], return_tensors = "pt").to("cuda")
    # Sinh phản hồi từ LLM

    outputs = model.generate(**inputs, streamer=text_streamer, max_new_tokens=256)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Lấy phần trả lời
    response_start = generated_text.find("### Response:") + len("### Response:")
    response = generated_text[response_start:].strip()

    return response

# Đường dẫn FAISS và metadata
faiss_path = "/content/vector_database.faiss"
metadata_path = "/content/metadata.csv"

# Load FAISS index và metadata
index, metadata = load_faiss_index(faiss_path, metadata_path)
retrieval_model = SentenceTransformer("intfloat/multilingual-e5-small")

Đang tải FAISS index và metadata...


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/498k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [ ]:
question = "Hợp đồng tương lai là gì?"
response = rag_llm_pipeline(question, model, retrieval_model, tokenizer, index, metadata, alpaca_prompt, k=2)

<|begin_of_text|>Instruction: Bạn là một chuyên gia tài chính, bạn được cung cấp những chỉ số tài chính của một mô hình dự đoán giá và những tài liệu tài chính kèm theo. Người dùng có thể hỏi về những chỉ số này hoặc những câu hỏi khác, nếu người dùng hỏi về chỉ số hoặc thông tin liên quan đến mô hình, hãy dùng số liệu được cung cấp để trả lời, còn nếu người dùng hỏi những câu hỏi khác không liên quan đến mô hình thì không cần dùng những chỉ số tài chính của mô hình.

### Context:
Phí giao dịch:Miễn phí
Thời gian thanh toán:Tối đa15 ngày làm việctừ ngày DNSE nhận được hồ sơ hợp lệ của khách hàng.
Cách thức giao dịch
Cách 1:Trực tiếp tại quầy
Cách 2:Đăng ký qua email hello@dnse.com.vn và ký vào hồ sơ bản cứng
(DNSE thực hiện thu mua cổ phiếu trên tiểu khoản thường: SpaceX/ Plus Cash/ Premier Cash. Vì vậy trước khi đăng ký bán cổ phiếu lẻ, KH cần chuyển cổ phiếu về 1 tài khoản)
Giao dịch cổ phiếu lô lẻ sàn HNX, Upcom
Sàn HNX: Sở giao dịch chứng khoán Hà Nội và đây cũng là sàn giao dịch l

In [ ]:
questions = list(test_data['Text'])
true_answers = list(test_data['Choose'])
model_rag_answer=[]

for i in range(len(questions)): #len(questions)
  response = rag_llm_pipeline(test_data[i], model, retrieval_model, tokenizer, index, metadata, alpaca_prompt, k=2)
  model_rag_answer.append(response)


Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
### Context:
Chỉ số của mô hình:
 Return (Ann.) [%]: 26.58,
 Volatility (Ann.) [%]: 8.08,
 Max. Drawdown [%]: -26.51,
 Sharpe Ratio: 3.29,
 Sortino Ratio: 2.07,
 Calmar Ratio: 1.0,
 Win Rate [%]: 50.16,
 AvgTrade [%]: 2.21,
 Generalization Score: -0.21,
 Tỉ lệ dự đoán gần đây [%]: 20. Phí giao dịch:Miễn phí
Thời gian thanh toán:Tối đa15 ngày làm việctừ ngày DNSE nhận được hồ sơ hợp lệ của khách hàng.
Cách thức giao dịch
Cách 1:Trực tiếp tại quầy
Cách 2:Đăng ký qua email hello@dnse.com.vn và ký vào hồ sơ bản cứng
(DNSE thực hiện thu mua cổ phiếu trên tiểu khoản thường: SpaceX/ Plus Cash/ Premier Cash. Vì vậy trước khi đăng ký bán cổ phiếu lẻ, KH cần chuyển cổ phiếu về 1 tài khoản)
Giao dịch cổ phiếu lô lẻ sàn HNX, Upcom
Sàn HNX: Sở giao dịch chứng khoán Hà Nội và đây cũng là sàn giao dịch lớn thứ 2
Tại Việt Nam hiện nay. Đây là một trong những nơi có mã cổ phiếu của các doanh nghiệp lớn và vừa.
Sàn UPCOM: giao dịch chứng khoán côn

In [ ]:
rag_results = evaluate_model(model_rag_answer, true_answers)
# Chuyển ngưỡng tương đồng thành nhãn nhị phân (giống hoặc không giống)
similarity_threshold = 0.5
binary_predictions = [1 if calculate_similarity(p, t) >= similarity_threshold else 0
                      for p, t in zip(model_rag_answer, true_answers)]
binary_true_answers = [1] * len(true_answers)

# Tính F1-score
f1_rag = f1_score(binary_true_answers, binary_predictions)
rag_results['f1']=f1
print("RAG Results:", rag_results)

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

RAG Results: {'bleu': 0.03545821290390997, 'rouge': {'rouge-1': {'r': 0.28555429483121575, 'p': 0.2073442043346577, 'f': 0.22240679832119192}, 'rouge-2': {'r': 0.1323618618000087, 'p': 0.09037029816902455, 'f': 0.09696811879777946}, 'rouge-l': {'r': 0.2600726113064677, 'p': 0.18820641537669316, 'f': 0.20145598531134812}}, 'f1': 0.10642201834862386}


In [ ]:
with open('evaluation_results_rag.json', 'w') as file: json.dump(rag_results, file)